In [1]:
from qcportal import PortalClient
from chargecraft.storage.qcarchive_transfer import QCArchiveToLocalDB
from qcportal import PortalClient
from qcportal.record_models import RecordQueryIterator
from chargecraft.storage.storage import MoleculePropRecord, MoleculePropStore
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.esp.qcresults import reconstruct_density, compute_esp
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Pool, get_context
from tqdm import tqdm
from openff.recharge.esp import DFTGridSettings
from chargecraft.storage.data_classes import ESPSettings, PCMSettings, DDXSettings
from qcportal.singlepoint.record_models import SinglepointRecord
import pickle
import psi4

The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html


In [2]:
client = PortalClient("api.qcarchive.molssi.org")

prop_store = MoleculePropStore("/mnt/storage/nobackup/nca121/QC_archive_50K_esp_gen/async_chargecraft/ESP_rebuilt.db")

grid_settings = LatticeGridSettings(
type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
)

records = client.query_records(
        record_id="96507636",
    )


In [3]:

qcarchdb = QCArchiveToLocalDB(
    qc_archive=client,
    prop_data_store=prop_store,
    grid_settings=grid_settings,
)

In [4]:
record = qcarchdb.process_item(item = next(records),
                    exclude_keys = None,
                    qm_esp = True,
                    compute_properties = True,
                    return_store = True
                    )

/mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4/chargecraft/storage/qcarchive_transfer.py:356: FutureWarning: Using QCVariable `MULLIKEN_CHARGES` instead of `MULLIKEN CHARGES` is deprecated, and as soon as 1.5 it will stop working

  variables_dictionary["MULLIKEN_CHARGES"] = psi4_wavefunction.variable("MULLIKEN_CHARGES") * unit.e
/mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4/chargecraft/storage/qcarchive_transfer.py:357: FutureWarning: Using QCVariable `LOWDIN_CHARGES` instead of `LOWDIN CHARGES` is deprecated, and as soon as 1.5 it will stop working

  variables_dictionary["LOWDIN_CHARGES"] = psi4_wavefunction.variable("LOWDIN_CHARGES") * unit.e


Computed variables dictionary
{'MULLIKEN_CHARGES': <Quantity([-0.16891819 -0.16888006 -0.16892575  0.06356222 -0.16892744 -0.16888092
  0.06348763 -0.16888877 -0.86013837 -0.86017656  0.28703742  0.28703913
  0.28703148  0.28703037  0.28703813  0.28703199  0.48074553  0.48074569
  0.48074561  0.48074694  0.48074702  0.48074692], 'elementary_charge')>, 'LOWDIN_CHARGES': <Quantity([-0.12052113 -0.11904267 -0.12077604  0.04497781 -0.11902806 -0.12079097
  0.04498213 -0.12053818 -0.32609156 -0.32612869  0.19438234  0.19253603
  0.19357599  0.19252861  0.19358281  0.19438444  0.35306208  0.35277376
  0.35513503  0.35278628  0.353061    0.35514897], 'elementary_charge')>, 'MBIS CHARGES': <Quantity([[-0.13809136]
 [-0.13809915]
 [-0.13816157]
 [ 0.22710067]
 [-0.13809164]
 [-0.13817015]
 [ 0.22710002]
 [-0.13810717]
 [-0.61156947]
 [-0.61156025]
 [ 0.17302684]
 [ 0.17303271]
 [ 0.17303041]
 [ 0.17302077]
 [ 0.17304127]
 [ 0.17303126]
 [ 0.42657288]
 [ 0.42656711]
 [ 0.42657259]
 [ 0.42656101]

/mnt/nfs/home/nca121/openff-recharge/openff/recharge/utilities/pydantic.py:33: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return numpy.array(val, dtype=dtype)


In [5]:

records: RecordQueryIterator = client.query_records(
        record_id="96507636",
    )

record: SinglepointRecord =  next(records)

density = reconstruct_density(wavefunction=record.wavefunction, n_alpha=record.properties['calcinfo_nalpha'])
dft_grid_settings = None
#HF will not contain this keyword, use default grid settings
dft_grid_settings = DFTGridSettings.Default


esp_settings = ESPSettings(
    basis = record.specification.basis,
    method = record.specification.method,
    grid_settings = grid_settings,
    #TODO update PCMSettings if use. Fix radii set and solvent choices in the database
    pcm_settings = PCMSettings(
        solver = '', 
        solvent = '',
        radii_model = '',
        radii_scaling = '',
        cavity_area = ''
    ) if 'PCM' in record.specification.keywords else None,
    ddx_settings = DDXSettings(
        solvent = None if not 'ddx_solvent_epsilon' in record.specification.keywords
        else record.specification.keywords['ddx_solvent'],
        epsilon = record.specification.keywords['ddx_solvent_epsilon'] 
        if 'ddx_solvent_epsilon' in record.specification.keywords is not None else None,
        radii_set = 'uff',
        ddx_model = record.specification.keywords['ddx_model'].upper() 
        if record.specification.keywords['ddx_model'] is not None else None)
        if 'ddx' in record.specification.keywords else None,
    psi4_dft_grid_settings = dft_grid_settings
)

qc_molecule = record.molecule

psi4.core.be_quiet()

psi4_molecule = psi4.geometry(qc_molecule.to_string("psi4", "angstrom"))
psi4_molecule.reset_point_group("c1")

psi4_wavefunction = psi4.core.RHF(
    psi4.core.Wavefunction.build(psi4_molecule, esp_settings.basis),
    psi4.core.SuperFunctional(),
)
wavefunction = psi4_wavefunction.Da().copy(psi4.core.Matrix.from_array(density))

psi4.oeprop(wavefunction,"DIPOLE",
                        "QUADRUPOLE", 
                        "MULLIKEN_CHARGES",
                        "LOWDIN_CHARGES",
                        "MBIS_CHARGES",
                        "MBIS_DIPOLE",
                        "MBIS_QUADRUPOLE")
            
# variables_dictionary = dict()
# #psi4 computes charges in a.u., elementary charge
# variables_dictionary["MULLIKEN_CHARGES"] = psi4_wavefunction.variable("MULLIKEN_CHARGES") * unit.e
# variables_dictionary["LOWDIN_CHARGES"] = psi4_wavefunction.variable("LOWDIN_CHARGES") * unit.e
# variables_dictionary["MBIS CHARGES"] = psi4_wavefunction.variable("MBIS CHARGES") * unit.e
# #psi4 grab the MBIS multipoles
# variables_dictionary["MBIS DIPOLE"] = psi4_wavefunction.variable("MBIS DIPOLES") * unit.e * unit.bohr_radius
# variables_dictionary["MBIS QUADRUPOLE"] = psi4_wavefunction.variable("MBIS QUADRUPOLES") * unit.e * unit.bohr_radius**2
# variables_dictionary["MBIS OCTOPOLE"] = psi4_wavefunction.variable("MBIS OCTUPOLES") * unit.e * unit.bohr_radius**3
# variables_dictionary["DIPOLE"] = psi4_wavefunction.variable("DIPOLE") * unit.e * unit.bohr_radius
# variables_dictionary["QUADRUPOLE"] = psi4_wavefunction.variable("QUADRUPOLE") * unit.e * unit.bohr_radius**2
# variables_dictionary["ALPHA_DENSITY"] = psi4_wavefunction.Da().to_array()
# variables_dictionary["BETA_DENSITY"] = psi4_wavefunction.Db().to_array()

: 

In [ ]:
pickle.dumps(record)

b'\x80\x04\x95\x82<\x01\x00\x00\x00\x00\x00\x8c\x1bchargecraft.storage.storage\x94\x8c\x12MoleculePropRecord\x94\x93\x94)\x81\x94}\x94(\x8c\x08__dict__\x94}\x94(\x8c\rtagged_smiles\x94\x8c\x9d[C:1]12([H:11])[C:2]3([H:12])[C:3]4([H:13])[C:4]1([N+:10]([H:20])([H:21])[H:22])[C:5]1([H:14])[C:6]2([H:15])[C:7]3([N+:9]([H:17])([H:18])[H:19])[C:8]41[H:16]\x94\x8c\tconformer\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x16K\x03\x86\x94h\r\x8c\x05dtype\x94\x93\x94\x8c\x02f8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x10\x02\x00\x00(.\xf7\xe6EK\xe0\xbf&\x12\xe1d\xf4\x08\xf4\xbf\n\x9b\x90U\xfd\xd7\xbc?,\xa1\xef\xb8?\r\xf0?\xce\xff\x93\x9fl%\xed\xbf\xdeAar\x1d\xd9\xaf\xbf\xa3\x12\x06e\xb4\x15\xe5?L\xd9\x1b\x8a\xfas\xbf?\x8e\n\xfd3\x94\xdd\xf2\xbf^\x8cC*\xd7\'\xeb\xbfc,\xaac\xf0\xcc\xcb\xbf\xc6\xb5\xcaw\xc6\xf2\xef\xbf2]w\xef\x14\r\xf0\x